# Predicting interactions with kNN

In [ ]:
import Pkg; Pkg.activate(".")
using Plots
using StatsBase

## Getting the data

We will get data from the *Ecological Interactions and the Netflix Problem* paper, which was assigned reading for this chapter. The data will go into a specific folder, which we create if it doesn't exist:

In [ ]:
# Create a folder called data if it doesn't exist
ispath(joinpath("data", "knn")) || mkdir(joinpath("data", "knn"))

All the data files are stored on *GitHub* - we can get them directly, using the `download` function. To save time, we only download them if they do not exist yet. The `joinpath` function will ensure that our code works on all operating systems:

In [ ]:
_project_root = "https://raw.githubusercontent.com/PhDP/EcoInter/master/KNN/data/"
isfile(joinpath("data", "knn", "interactions.csv")) || download(_project_root*"mercure-interactions.csv", joinpath("data", "knn", "interactions.csv"))
isfile(joinpath("data", "knn", "traits-codes.txt")) || download(_project_root*"mercure-trait-codes.txt", joinpath("data", "knn", "traits-codes.txt"))
isfile(joinpath("data", "knn", "traits.csv")) || download(_project_root*"mercure-traits.csv", joinpath("data", "knn", "traits.csv"))

## Predictions based only on interactions

In the first part, we will make a prediction based only on interactions. Specifically, we will try to estimate the amount of information needed to make a prediction, by building a "recommender", *i.e.* an algorithm that will suggest a prey to a predator, based on what other predators with similar diets are eating. To do this, we will store the diet of every predator as a `Set`:

In [ ]:
data_interactions = joinpath("data", "knn", "interactions.csv")
preys = Set{Int64}[]

We can now fill the `preys` array, by reading all of the lines (have a look at the `interactions.csv` file to see how it is organized):

In [ ]:
for (i,line) in enumerate(readlines(data_interactions))
    if length(line) > 0
        this_preys = parse.(Int64, split(line, ", "))
        push!(preys, Set(this_preys))
    else
        push!(preys, Set(Int64[]))
    end
end

Let's look at what the first 5 predators are eating:

In [ ]:
preys[1:5]

To get an idea of which preys have "the same diet", we will use the Tanimoto distance, which is the cardinality of the intersect of the diets divided by the cardinality of their union:

In [ ]:
tanimoto(s1::T, s2::T) where {T <: Set} = length(s1 ∩ s2)/length(s1 ∪ s2)

The Tanimoto measure reflects similarity, and is bounded by 0 and 1, so we can simply substract it to unity to get a distance measure:

In [ ]:
D(s1,s2) = 1.0 - tanimoto(s1,s2)

In [ ]:
function success_rate_int_only(preys; k::Int64=3, top::Int64=5)
    @assert 1 ≤ top ≤ 20
    @assert 1 ≤ k ≤ (length(preys)-1)
    attempts = zeros(Bool, length(preys))
    successes = zeros(Bool, length(preys))
    for prey_idx in 1:length(preys)
        if length(preys[prey_idx]) > 2
            sampled_preys = sample(collect(preys[prey_idx]), length(preys[prey_idx])-1, replace=false)
            removed_prey = first(collect(filter(p -> !(p in sampled_preys), preys[prey_idx])))
            test_preys = preys[filter(i -> i != prey_idx, 1:length(preys))]
            neighbors = [D(Set(sampled_preys), p) for p in test_preys]
            likely_preys = test_preys[partialsortperm(neighbors, 1:k)]
            recos = countmap(vcat(collect.(likely_preys)...))
            recommended = zeros(Bool, top)
            for re in 1:top
                if length(recos) > 0
                    cmax = filter(f -> f.second == maximum(values(recos)), recos)
                    recommended[re] = removed_prey in collect(keys(cmax))
                    filter!(f -> f.second < maximum(values(recos)), recos)
                end
            end
            successes[prey_idx] = sum(recommended)
            attempts[prey_idx] = true
        end
    end
    return sum(successes)/sum(attempts)
end

In [ ]:
k = 1:1:7
r = [success_rate_int_only(preys; k=x, top=5) for x in k]
scatter(k, r, lab="Top 5")
yaxis!((0.5,1), "Success rate")
xaxis!("k")

## Adding traits

In [ ]:
data_traits = joinpath("data", "knn", "traits.csv")
traits = Set{Int64}[]
for (i,line) in enumerate(readlines(data_traits))
    if length(line) > 0
        this_traits = parse.(Int64, split(line, ", "))
        push!(traits, Set(this_traits))
    else
        push!(traits, Set(Int64[]))
    end
end

In [ ]:
function success_rate_with_traits(preys, traits; k::Int64=3, top::Int64=5, wt::Float64=0.5)
    @assert 1 ≤ top ≤ 20
    @assert 1 ≤ k ≤ (length(preys)-1)
    @assert 0.0 <= wt <= 1.0
    attempts = zeros(Bool, length(preys))
    successes = zeros(Bool, length(preys))
    for prey_idx in 1:length(preys)
        if length(preys[prey_idx]) > 2
            sampled_preys = sample(collect(preys[prey_idx]), length(preys[prey_idx])-1, replace=false)
            removed_prey = first(collect(filter(p -> !(p in sampled_preys), preys[prey_idx])))
            test_preys = preys[filter(i -> i != prey_idx, 1:length(preys))]
            test_traits = traits[filter(i -> i != prey_idx, 1:length(traits))]
            neighbors = [(1-wt)*D(Set(sampled_preys), test_preys[i])+wt*D(traits[prey_idx], test_traits[i]) for i in 1:length(test_preys)]
            likely_preys = test_preys[partialsortperm(neighbors, 1:k)]
            recos = countmap(vcat(collect.(likely_preys)...))
            recommended = zeros(Bool, top)
            for re in 1:top
                if length(recos) > 0
                    cmax = filter(f -> f.second == maximum(values(recos)), recos)
                    recommended[re] = removed_prey in collect(keys(cmax))
                    filter!(f -> f.second < maximum(values(recos)), recos)
                end
            end
            successes[prey_idx] = sum(recommended)
            attempts[prey_idx] = true
        end
    end
    return sum(successes)/sum(attempts)
end

In [ ]:
success_rate_int_only(preys; k=4, top=3)

In [ ]:
success_rate_with_traits(preys, traits; k=4, top=3, wt=0.0)

In [ ]:
wt = 0.0:0.1:1.0
r = [success_rate_with_traits(preys, traits; k=4, top=3, wt=x) for x in wt]
scatter(wt, r, leg=false)
yaxis!((0.5,1), "Success rate")
xaxis!("Relative importance of traits")

In [ ]:
plot(wt, r , leg=false)